### Домашняя работа

**Задание простого уровня** Мы говорили, что метрики качества нужны, чтобы сравнивать различные модели между собой. В задаче полиномиальной регрессии реализуйте код для выбора лучшей степени полиному:

* возьмите все степени от 1 до 10 по порядку, без пропусков.
* найдите степень полинома, где будет лучший r2-score
* напишите код, который выводит самую подходящую степень полинома и соответствующий ей скор

Эта процедура называется Grid Search и помогает найти лучшие параметры для модели.

Обучите лучшую модель и сделайте predict

In [3]:
import pandas as pd
import numpy as np

data = pd.read_csv('non_linear.csv', sep=',')

features = data.x_train
target = data.y_train

data.head()

,x_train,y_train
0,0.138368,0.838812
1,0.157237,0.889313
2,0.188684,1.430040
3,0.685553,1.717309
4,0.874237,2.032588


In [12]:
target.shape

(506,)

In [4]:
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

def grid_search(features, target):
  r_score = 0
  best_degree = 0
  for n in range(1, 11):
    transformed_feature = np.array([features**i for i in range(1, n+1)]).T
    reg = LinearRegression().fit(transformed_feature, target)
    predicted_target = reg.predict(transformed_feature)
    temp_r = r2_score(target, predicted_target)
    print('degree = %.d' % (n))
    print('r2 score = %.4f\n' % (temp_r))
    if temp_r > r_score:
      r_score = temp_r
      best_degree = n
  return best_degree, r_score

In [5]:
best_d, best_score = grid_search(features, target)

degree = 1
r2 score = 0.5195

degree = 2
r2 score = 0.5335

degree = 3
r2 score = 0.8884

degree = 4
r2 score = 0.8955

degree = 5
r2 score = 0.8956

degree = 6
r2 score = 0.8993

degree = 7
r2 score = 0.9005

degree = 8
r2 score = 0.9087

degree = 9
r2 score = 0.9088

degree = 10
r2 score = 0.9091



In [6]:
print(best_d, best_score)

10 0.9091133831300623


**Задание среднего уровня** Напишите класс для обучения модели, который содержит:

* функцию `.fit(X, y)` , которая принимает на вход массив фичей `X`, массив таргетов `y` и обучает коэффициенты регрессии. Код для обучения взять из первого урока модуля *Постановка ML задачи линейной регрессии*
* функцию `.predict(X)`, которая по массиву фичей `X` возвращает массив предсказаний `y`

Нужно использовать код для аналитически вычисляемых коэффициентов.

Это задание позволит понять, как работает линейная регрессия "внутри" библиотечной реализации.

In [7]:
from numpy.linalg import inv

class CustomLinearReg:
    def __init__(self):
        pass

    def fit(self, X, y):
        # pass
        new_X = np.array([[1, i] for i in X])
        new_y = np.array(y).reshape(-1,1)
        self.w = inv(new_X.T.dot(new_X)).dot(new_X.T.dot(new_y))

    def predict(self, X):
        # pass
        w = self.w
        predicted_y = w[0][0] + w[1][0]*X
        return predicted_y


In [89]:
target[1]

0.889313260737732

In [8]:
reg = CustomLinearReg()
reg.fit(features, target)
reg.predict(features[1])

1.9163115681550749

**Задание высокого уровня**

1. разделите датасет с домами Бостона из Урока 2 (таргет и фичи) на две части: в одной части 80% датасета (назовём train) в другой 20% (назовём valid) с помощью функции `train_test_split` из библиотеки `sklearn`
1. обучите модель только на train датасете
1. постройте предсказания valid датасете
1. Посчитайте  `r2 score` на валидационном сете

После этого примените к обеим датасетам z-преобразование и повторите шаги 2-4. Как изменилась метрика r2?

Это задание поможет понять, как валидировать линейную регрессию (и другие модели) на отложенной выборке.

In [118]:
raw_df = pd.read_csv('housing.csv', sep="\s+", skiprows=22, header=None)
features_2 = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target_2 = raw_df.values[1::2, 2]

In [98]:
# raw_df = pd.read_csv('housing.csv', sep="\s+", skiprows=22, header=None)
# raw_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.23247,0.0,8.14,0,0.538,6.142,91.7,3.9769,4,307.0,21.0,396.90,18.72,15.2
1,0.98843,0.0,8.14,0,0.538,5.813,100.0,4.0952,4,307.0,21.0,394.54,19.88,14.5
2,0.75026,0.0,8.14,0,0.538,5.924,94.1,4.3996,4,307.0,21.0,394.33,16.30,15.6
3,0.84054,0.0,8.14,0,0.538,5.599,85.7,4.4546,4,307.0,21.0,303.42,16.51,13.9
4,0.67191,0.0,8.14,0,0.538,5.813,90.3,4.6820,4,307.0,21.0,376.88,14.81,16.6


In [124]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(features_2, target_2, test_size=0.2, random_state = 42)

In [131]:
x_valid.shape

(49, 16)

In [136]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
reg = LinearRegression().fit(x_train, y_train)
y_pred = reg.predict(x_valid)

In [137]:
y_pred.shape

(49,)

In [151]:
r2_first = r2_score(y_valid, y_pred)
print('r2 score for current data = %.5f' % r2_first)

r2 score for current data = 0.97596


In [141]:
from sklearn.preprocessing import StandardScaler

In [147]:
transformed_x_train = StandardScaler().fit_transform(x_train)
transformed_x_valid = StandardScaler().fit_transform(x_valid)

In [149]:
reg_2 = LinearRegression().fit(transformed_x_train, y_train)
y_2_pred = reg_2.predict(transformed_x_valid)

In [152]:
r2_transformed = r2_score(y_valid, y_2_pred)
print('new r2 score for transformed data = %5f' % r2_transformed)

new r2 score for transformed data = 0.937884
